# Objeto DataFrame
---

## Crear SparkContext y SparkSession

In [1]:
# Crear puntos de entrada para spark
try:
    sc.stop()
except:
    pass
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
sc=SparkContext()
spark = SparkSession(sparkContext=sc)

## Crear objeto DataFrame

### Crear DataFrame leyendo un archivo

In [2]:
mtcars = spark.read.csv(path='../../data/mtcars.csv',
                        sep=',',
                        encoding='UTF-8',
                        comment=None,
                        header=True, 
                        inferSchema=True)
mtcars.show(n=5, truncate=False)

+-----------------+----+---+-----+---+----+-----+-----+---+---+----+----+
|_c0              |mpg |cyl|disp |hp |drat|wt   |qsec |vs |am |gear|carb|
+-----------------+----+---+-----+---+----+-----+-----+---+---+----+----+
|Mazda RX4        |21.0|6  |160.0|110|3.9 |2.62 |16.46|0  |1  |4   |4   |
|Mazda RX4 Wag    |21.0|6  |160.0|110|3.9 |2.875|17.02|0  |1  |4   |4   |
|Datsun 710       |22.8|4  |108.0|93 |3.85|2.32 |18.61|1  |1  |4   |1   |
|Hornet 4 Drive   |21.4|6  |258.0|110|3.08|3.215|19.44|1  |0  |3   |1   |
|Hornet Sportabout|18.7|8  |360.0|175|3.15|3.44 |17.02|0  |0  |3   |2   |
+-----------------+----+---+-----+---+----+-----+-----+---+---+----+----+
only showing top 5 rows



### Crear DataFrame con la funcion `createDataFrame`

#### Desde un RDD

Los elementos en RDD tienen que ser un objeto Row

In [3]:
from pyspark.sql import Row
rdd = sc.parallelize([
    Row(x=[1,2,3], y=['a','b','c']),
    Row(x=[4,5,6], y=['e','f','g'])
])
rdd.collect()

[Row(x=[1, 2, 3], y=['a', 'b', 'c']), Row(x=[4, 5, 6], y=['e', 'f', 'g'])]

In [4]:
df = spark.createDataFrame(rdd)
df.show()

+---------+---------+
|        x|        y|
+---------+---------+
|[1, 2, 3]|[a, b, c]|
|[4, 5, 6]|[e, f, g]|
+---------+---------+



#### Con la libreria pandas DataFrame

In [5]:
import pandas as pd
pdf = pd.DataFrame({
    'x': [[1,2,3], [4,5,6]],
    'y': [['a','b','c'], ['e','f','g']]
})
pdf

,x,y
0,"[1, 2, 3]","[a, b, c]"
1,"[4, 5, 6]","[e, f, g]"


In [7]:
df = spark.createDataFrame(pdf)
df.show()

+---------+---------+
|        x|        y|
+---------+---------+
|[1, 2, 3]|[a, b, c]|
|[4, 5, 6]|[e, f, g]|
+---------+---------+



#### De una lista

Cada elemento de la lista se convierte en una Fila en el DataFrame. 

In [8]:
my_list = [['a', 1], ['b', 2]]
df = spark.createDataFrame(my_list, ['letter', 'number'])
df.show()

+------+------+
|letter|number|
+------+------+
|     a|     1|
|     b|     2|
+------+------+



In [9]:
df.dtypes

[('letter', 'string'), ('number', 'bigint')]

In [10]:
my_list = [['a', 1], ['b', 2]]
df = spark.createDataFrame(my_list, ['my_column'])
df.show()

+---------+---+
|my_column| _2|
+---------+---+
|        a|  1|
|        b|  2|
+---------+---+



In [11]:
df.dtypes

[('my_column', 'string'), ('_2', 'bigint')]

El siguiente código genera un DataFrame que consiste en dos columnas, cada columna es una columna vectorial.

¿Por qué se generan columnas vectoriales en este caso?
En este caso, la lista **my_list** tiene sólo un elemento, una tupla. Por lo tanto, el DataFrame sólo tiene una fila. Esta tupla tiene dos elementos. Por lo tanto, genera un DataFrame de dos columnas. Cada elemento de la tupla es una lista, por lo que las columnas resultantes son columnas vectoriales.

In [12]:
my_list = [(['a', 1], ['b', 2])]
df = spark.createDataFrame(my_list, ['x', 'y'])
df.show()

+------+------+
|     x|     y|
+------+------+
|[a, 1]|[b, 2]|
+------+------+





## Instancia Column

Las instancias de columna se pueden crear de dos maneras:

1. Seleccione directamente una columna de entre un *DataFrame*: `df.colName`
2. crear a partir de una expresión de columna: `df.colName + 1`

Técnicamente, sólo hay una forma de crear una instancia de columna. Las expresiones de columna comienzan desde una instancia de columna.
**Recuerde cómo crear instancias de columna, ya que éste es normalmente el punto de partida si queremos operar con columnas DataFrame.**

Las clases de columna vienen con algunos métodos que pueden operar en una instancia de columna. ***Sin embargo, casi todas las funciones de `pyspark.sql.functions` tomar una o más instancias de columna como argumentos***. Estas funciones son importantes para las herramientas de manipulación de datos.

## DataFrame metodos column

### Methods that take column names as arguments:

* `corr(col1, col2)`: nombres de dos columnas.
* `cov(col1, col2)`: nombres de dos columnas.
* `crosstab(col1, col2)`: nombres de dos columnas.
* `describe(*cols)`: ***`*cols` se refiere sólo a nombres de columna (strings).***

### Métodos que toman nombres de columnas o expresiones de columnas o **both** como argumentos:

* `cube(*cols)`: nombres de columna (string) o expresiones de columna o **both**.
* `drop(*cols)`: ***una lista de nombres de columnas O una expresión de una sola columna.***
* `groupBy(*cols)`: nombre de columna (string) o expresión de columna o **both**.
* `rollup(*cols)`: nombre de columna (string) o expresión de columna o **both**.
* `select(*cols)`: nombre de columna (string) o expresión de columna o **both**.
* `sort(*cols, **kwargs)`: column name (string) or column expression or **both**.
* `sortWithinPartitions(*cols, **kwargs)`: nombre de columna (string) o expresión de columna o **both**.
* `orderBy(*cols, **kwargs)`: nombre de columna (string) o expresión de columna o **both**.
* `sampleBy(col, fractions, sed=None)`: nombre de columna.
* `toDF(*cols)`: **una lista de nombres de columnas (string).**
* `withColumn(colName, col)`: `colName` se refiere al nombre de la columna; `col` se refiere a una expresión de columna.
* `withColumnRenamed(existing, new)`: toma los nombres de las columnas como argumentos.
* `filter(condition)`: ***condicion** se refiere a una expresión de columna que devuelve `types.BooleanType` de valores. 